In [1]:
from config_paths import *
import sys
sys.path.append(str(SRC_DIR))

from fuzzy_sql.fuzzy_sql import *
import json
import os
from pathlib import Path

DATASET_NAME='cms'

In [2]:
# set directories
metadata_dir = os.path.join(DATA_DIR, DATASET_NAME,'processed/metadata')
real_dir = os.path.join(DATA_DIR,DATASET_NAME, 'raw/sample1')
syn_dir = os.path.join(DATA_DIR,DATASET_NAME, 'raw/sample2')
db_path = os.path.join(DB_DIR, f'{DATASET_NAME}.db')

In [3]:
# identify input data file names
real_csvs=["DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv",\
    "DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv",\
    "DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv",\
    "DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.csv",\
    "DE1_0_2008_to_2010_Carrier_Claims_Sample_1B.csv" ,\
    "DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv",\
    "DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv",\
    "DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv" ]

meta_jsons=['s1_ben_sum_2008.json','s1_ben_sum_2009.json','s1_ben_sum_2010.json','s1_carrier_1a.json',\
    's1_carrier_1b.json','s1_inpatient.json','s1_outpatient.json','s1_prescrp.json']

syn_csvs=["DE1_0_2008_Beneficiary_Summary_File_Sample_2.csv",\
    "DE1_0_2009_Beneficiary_Summary_File_Sample_2.csv",\
    "DE1_0_2010_Beneficiary_Summary_File_Sample_2.csv",\
    "DE1_0_2008_to_2010_Carrier_Claims_Sample_2A.csv",\
    "DE1_0_2008_to_2010_Carrier_Claims_Sample_2B.csv" ,\
    "DE1_0_2008_to_2010_Inpatient_Claims_Sample_2.csv",\
    "DE1_0_2008_to_2010_Outpatient_Claims_Sample_2.csv",\
    "DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_2.csv" ]

#extracted rows from corresponding tables
rows=[10000,10000,10000,20000,20000,15000,15000,15000]

### Prepare data and import it into the database with indexing


In [4]:
conn = sqlite3.connect(db_path)
with conn:
    for real_csv,meta_json,syn_csv, rows_i in zip(real_csvs,meta_jsons,syn_csvs, rows):
        #Import real
        data,_=prep_data_for_db(os.path.join(real_dir,real_csv),nrows=rows_i) # Note: This function  can be used to quickly generate metadata template, but this is not shown here.
        with open(os.path.join(metadata_dir, meta_json), 'r') as f:
            metadata=json.load(f)
        candidate_db_idx=get_vars_to_index(metadata,data)
        table_name=Path(meta_json).stem
        make_table(table_name, data, conn,indx_vars=candidate_db_idx)

        #Import syn
        data,_=prep_data_for_db(os.path.join(syn_dir,syn_csv),nrows=rows_i) # Note: This function  can be used to quickly generate metadata template, but this is not shown here.
        table_name=meta_json.replace("s1","s2")
        table_name=Path(table_name).stem
        make_table(table_name, data, conn,indx_vars=candidate_db_idx)

Table s1_ben_sum_2008 is created in the database
.... The index: IDX_s1_ben_sum_2008_PPPYMT_OP is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_SP_STRKETIA is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_BENRES_CAR is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_BENE_DEATH_DT is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_BENE_RACE_CD is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_BENRES_OP is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_MEDREIMB_IP is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_SP_ALZHDMTA is created for the table: s1_ben_sum_2008 in the database
.... The index: IDX_s1_ben_sum_2008_BENE_HI_CVRAGE_TOT_MONS is created for the table: s1_ben_sum_2008 in 